In [94]:
import time
import math
import uuid
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import joblib
import talib
import ta
import ccxt  # rea

In [95]:
exchange = ccxt.binance({
    'enableRateLimit': True,
})


In [96]:
symbol = "BTC/USDT"
market_type = "spot"           # 'spot' implemented here. For futures you'd change code.
model_timeframe = "15m"
price_timeframe = "1s"
limit = 300
window_size = 10

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

SAVED_MODEL = "greg_tech_5.pth"
SCALER_FILE = "scaler_15m.pkl"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [97]:
# -------------------------
# MODEL
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=4, output_dim=3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=device))
model.to(device); model.eval()
scaler = joblib.load(SCALER_FILE)


In [98]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df


In [99]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [100]:
import ta
def compute_indicators(df):
    out = {}
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    sma_window = 3
    out['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)
    out['TSI'] = compute_tsi(df['close'])
    period = 14
    smooth_k = 3
    smooth_d = 3

    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()

    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    
    feat_df = pd.DataFrame(out, index=df.index)
    return feat_df


In [101]:
# -------------------------
# Prediction
# -------------------------
hold_factor = 0.2  # Adjust this factor to increase/decrease hold probability
def predict_label_from_window(window_features):
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        probs[1] *= hold_factor
        probs = probs / probs.sum()
        pred = int(np.argmax(probs))
    return pred  # 0=down 1=hold 2=up


In [102]:
def find_swings(df, length=9):
    """
    Identify swing highs and lows exactly like TradingView's ta.highest()/lowest() confirmation.
    """
    swing_highs = []
    swing_lows = []

    for i in range(len(df)):
        if i < length or i > len(df) - length - 1:
            swing_highs.append(False)
            swing_lows.append(False)
            continue

        # TV logic: candle is swing high if its high is the highest in [i-length, i]
        window_high = df['high'].iloc[i - length:i + 1].max()
        window_low = df['low'].iloc[i - length:i + 1].min()

        swing_highs.append(df['high'].iloc[i] == window_high)
        swing_lows.append(df['low'].iloc[i] == window_low)

    df['swing_high'] = swing_highs
    df['swing_low'] = swing_lows
    return df


In [103]:
def detect_order_blocks_and_msb(df, length=9, fib_factor=0.33):
    """
    Full TradingView-style MSB + OB detector.
    Returns bullish/bearish OB zones and MSB direction.
    """
    df = find_swings(df.copy(), length)
    ob_zones = {'bullish_ob': None, 'bearish_ob': None}
    msb = 'none'
    trend = None

    last_swing_high = None
    last_swing_low = None

    for i in range(len(df)):
        # Record new swings
        if df['swing_high'].iloc[i]:
            last_swing_high = df['high'].iloc[i]
        if df['swing_low'].iloc[i]:
            last_swing_low = df['low'].iloc[i]

        # Skip until we have both
        if last_swing_high is None or last_swing_low is None:
            continue

        # Check bullish MSB (close above last swing high)
        if df['close'].iloc[i] > last_swing_high:
            msb = 'bullish'
            trend = 1
            # Bullish OB: last bearish candle before breakout
            prev_red = df.iloc[:i][ (df.iloc[:i]['close'] < df.iloc[:i]['open']) ]
            if not prev_red.empty:
                red_idx = prev_red.index[-1]
                ob_zones['bullish_ob'] = (
                    df.loc[red_idx, 'low'],
                    df.loc[red_idx, 'high']
                )

        # Check bearish MSB (close below last swing low)
        elif df['close'].iloc[i] < last_swing_low:
            msb = 'bearish'
            trend = -1
            # Bearish OB: last bullish candle before breakdown
            prev_green = df.iloc[:i][ (df.iloc[:i]['close'] < df.iloc[:i]['open']) ]
            if not prev_green.empty:
                green_idx = prev_green.index[-1]
                ob_zones['bearish_ob'] = (
                    df.loc[green_idx, 'low'],
                    df.loc[green_idx, 'high']
                )

    return {'bullish_ob': ob_zones['bullish_ob'],
            'bearish_ob': ob_zones['bearish_ob'],
            'msb': msb,
            'trend': trend}


In [104]:
df_ohlcv = fetch_latest_ohlcv(symbol, model_timeframe, limit)
result = detect_order_blocks_and_msb(df_ohlcv)
print(result)
feat_df = compute_indicators(df_ohlcv).dropna()
pred = predict_label_from_window(feat_df[FEATURES].values[-window_size:])
print()
print(pred)


{'bullish_ob': (np.float64(121420.0), np.float64(121584.77)), 'bearish_ob': None, 'msb': 'bullish', 'trend': 1}

0


In [105]:
def generate_trade_signal(symbol, model_timeframe, limit, window_size=10):
    """
    Combines ML prediction + TV-style MSB/OB structure.
    """
    df = fetch_latest_ohlcv(symbol, model_timeframe, limit)
    ind_df = compute_indicators(df).dropna()
    feat_df = df.join(ind_df, how='inner').dropna()

    ml_signal = predict_label_from_window(feat_df[FEATURES].values[-window_size:])
    ob_msb = detect_order_blocks_and_msb(feat_df)

    last_close = float(df['close'].iloc[-1])

    # BUY logic
    if ml_signal == 2:  # uptrend
        if ob_msb['bearish_ob']:
            low, high = ob_msb['bearish_ob']
            if low <= last_close <= high:
                return "buy"
        elif ob_msb['msb'] == 'bullish':
            return "buy"

    # SELL logic
    elif ml_signal == 0:  # downtrend
        if ob_msb['bullish_ob']:
            low, high = ob_msb['bullish_ob']
            if low <= last_close <= high:
                return "sell"
        elif ob_msb['msb'] == 'bearish':
            return "sell"

    return "hold"


In [106]:
print(generate_trade_signal(symbol, model_timeframe, limit, window_size))

hold


In [107]:
def last_confirmed_reversal_full_df(symbol, model_timeframe, limit, window_size=10):
    """
    Determine the last confirmed reversal using the full dataframe.
    Returns:
        last_index, confirmed_signal (2=uptrend, 0=downtrend)
    """
    # 1️⃣ Fetch OHLCV data
    df_ohlcv = fetch_latest_ohlcv(symbol, model_timeframe, limit)

    # 2️⃣ Compute indicators
    ind_df = compute_indicators(df_ohlcv).dropna()
    feat_df = df_ohlcv.join(ind_df, how='inner').dropna()

    # 3️⃣ Run ML predictions with sliding window
    preds = []
    for i in range(window_size, len(feat_df)):
        window = feat_df[FEATURES].values[i - window_size:i]
        pred = predict_label_from_window(window)  # 0=down,1=hold,2=up
        preds.append(pred)

    # Align predictions with feat_df
    feat_df = feat_df.iloc[window_size:]
    feat_df['ml_pred'] = preds

    # 4️⃣ Detect OB and MSB for the full dataframe
    ob_msb = detect_order_blocks_and_msb(feat_df)

    # 5️⃣ Loop in reverse to find last confirmed reversal
    last_index = None
    confirmed_signal = None

    for idx in reversed(feat_df.index):
        ml_signal = feat_df.loc[idx, 'ml_pred']

        # Uptrend confirmation: ML says up + price in bearish OB or MSB bullish
        if ml_signal == 2:
            if (ob_msb['bearish_ob'] and ob_msb['bearish_ob'][0] <= feat_df.loc[idx, 'close'] <= ob_msb['bearish_ob'][1]) \
               or ob_msb['msb'] == 'bullish':
                last_index = idx
                confirmed_signal = 2
                break

        # Downtrend confirmation: ML says down + price in bullish OB or MSB bearish
        elif ml_signal == 0:
            if (ob_msb['bullish_ob'] and ob_msb['bullish_ob'][0] <= feat_df.loc[idx, 'close'] <= ob_msb['bullish_ob'][1]) \
               or ob_msb['msb'] == 'bearish':
                last_index = idx
                confirmed_signal = 0
                break

    return last_index, confirmed_signal


In [108]:
print(last_confirmed_reversal_full_df(symbol, model_timeframe, limit, window_size))

(Timestamp('2025-10-10 10:30:00'), 2)
